In [10]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [11]:
driver = webdriver.Chrome("C:/Users/Lenovo/chromedriver.exe")

In [12]:
url = 'https://www.amazon.com'
driver.get(url)

In [14]:

def get_url(search_term):
    template = 'https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Daps&field-keywords={}'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page{}'
    return url

In [15]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url ='https://www.amazon.com' + atag.get('href')
    try:
        price_parent = item.find('span', 'a-price')
        price =price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''

    result = (description, price, rating, review_count, url)
    return result

In [25]:
def main(search_term):
    driver = webdriver.Chrome("C:/Users/Lenovo/chromedriver.exe")
    records = []
    url = get_url(search_term)
    
    
    for page in range(1,3):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'class': 'a-section a-spacing-medium'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    with open('amazon_lenovo.csv', 'w', newline ='', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'CustomerCount', 'url'])
        writer.writerows(records)

In [19]:
main('apple laptop')

In [22]:
main('dell laptop')

In [24]:
main('hp laptop')

In [26]:
main('lenovo laptop')